# E2: Cross Validation for Hyperparameters of Uninformed Label Model

In [ ]:
from sklearn.model_selection import KFold
from snorkel.labeling.model import LabelModel
import numpy as np 
import itertools

In [ ]:
# Parameters
seed = None
n_folds = 5
metrics = ["accuracy", "f1", "coverage"]

In [ ]:
# Hyperparameters to experiment with
n_epochs = [100, 500, 1000, 5000]
lr = 10 ** (-1 * np.arange(1, 6, dtype=float))
l2 = [0.0, 0.01, 0.1]
optimizer = ["sgd","adam","adamax"]
lr_scheduler = ["constant","linear","exponential","step"]

h_params_all = list( itertools.product(n_epochs, lr, optimizer, lr_scheduler) )

In [ ]:
# Extract relevant parts of label matrix
lf_subset = list(range(57)) 
L_data = np.copy(L_alarms[:,lf_subset])
Y_data = alarms_df.true_label.values

In [ ]:
cv_results = np.zeros((len(list(h_params_all)), len(metrics)))

kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)

for train_idx, test_idx in kf.split(L_data):
    # Define training dataset
    L_train = L_data[train_idx]
    Y_train = Y_data[train_idx]
    # Define test dataset
    L_test = L_data[test_idx]
    Y_test = Y_data[test_idx]

    for i, h_params in enumerate(h_params_all):
        # Fit a Label Model
        l_model = LabelModel(cardinality=2, verbose=True)
        l_model.fit(L_train, n_epochs=h_params[0], lr=h_params[1], optimizer=h_params[2], lr_scheduler=h_params[3])

        # Evaluate model on test data
        scores = l_model.score(L_test, Y_test, metrics=metrics)
        cv_results[i,:] = cv_results[i,:] + list(scores.values())

In [ ]:
# Average the metrics
cv_results /= n_folds
print(cv_results)